In [1]:
%%bash
python3 -m pip install influxdb

In [2]:
from influxdb import InfluxDBClient

client = InfluxDBClient('10.10.139.45', 8086, 'root', 'root', 'example')
client.create_database('example')

def process(row):
    client.write_points([{
            "measurement": "clicks",
            "tags": {
                "visitor_browser": row['visitor_browser'],
                "visitor_country": row['visitor_country'],
            },
            "time": row['ts_ingest'],
            "fields": {
                "visitor_platform": row['visitor_platform'],
                "article_title": row['article_title'],
                "article": row['article'],
                "visitor_os": row['visitor_os'],
                "visitor_page_timer": row['visitor_page_timer'],
                "visitor_page_height": row['visitor_page_height'],
            },
        }])


In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.3 pyspark-shell'

import pyspark 
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

sc = SparkContext()
sc.setLogLevel("WARN")
spark = SparkSession(sc)


df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers","10.10.139.63:9092") \
    .option("subscribe", "testing") \
    .option("startingOffsets", "earliest") \
    .load()

schema = StructType([
    StructField("visitor_platform", StringType()),
    StructField("ts_ingest", LongType()),
    StructField("article_title", StringType()),
    StructField("visitor_country", StringType()),
    StructField("visitor_os", StringType()),
    StructField("article", StringType()),
    StructField("visitor_browser", StringType()),
    StructField("visitor_page_timer", IntegerType()),
    StructField("visitor_page_height", IntegerType()),
])

print(df.schema)

dfs = df.selectExpr("CAST(value AS STRING)") \
      .select(from_json(col("value"), schema) \
      .alias("json"))

df_data = dfs.select("json.*")

def foreach_batch_function(df, epoch_id):
    pass

# Debug in console
# query = df_data.writeStream.outputMode("append").format("console").start()
# query.awaitTermination()

# query = df_data.writeStream.foreachBatch(foreach_batch_function).start()
query = df_data.writeStream.foreach(process).start()
query.awaitTermination()


StructType(List(StructField(key,BinaryType,true),StructField(value,BinaryType,true),StructField(topic,StringType,true),StructField(partition,IntegerType,true),StructField(offset,LongType,true),StructField(timestamp,TimestampType,true),StructField(timestampType,IntegerType,true)))
